# 임베딩(Embedding)
- **임베딩은 Retrieval-Augmented Generation(RAG) 시스템의 세 번째 단계로, 문서분할 단계에서 생성된 문서 단위들을 기계가 이해할 수 있는 수치적 형태로 변환하는 과정**
- **문서의 의미를 벡터(숫자의 배열) 형태로 표현함으로써, 사용자가 입력한 질문(Query) 에 대하여 DB 에 저장한 문서 조각/단락(Chunk) 을 검색하여 가져올 때 유사도 계산시 활용**

<br>

### 임베딩의 필요성
- **의미 이해**: 자연어는 매우 복잡하고 다양한 의미를 내포. **임베딩을 통해 이러한 텍스트를 정량화된 형태로 변환함으로써, 컴퓨터가 문서의 내용과 의미를 더 잘 이해하고 처리**
- **정보 검색 향상**: 수치화된 벡터 형태로의 변환은 문서 간의 유사성을 계산하는 데 있어 필수적. 이는 관련 문서를 검색하거나, 질문에 가장 적합한 문서를 찾는 작업을 용이

<br>

<img src='https://wikidocs.net/images/page/233777/capture-20240612-004924.png' width=600>

<br>

### 임베딩된 단락 활용 예시

<img src='https://wikidocs.net/images/page/233777/capture-20240612-005049.png' width=600>

- 1번 단락: [0.1, 0.5, 0.9, ... , 0.1, 0.2]
- 2번 단락: [0.7, 0.1, 0.3, ... , 0.5, 0.6]
- 3번 단락: [0.9, 0.4, 0.5, ... , 0.4, 0.3]

<br>

> **질문: "시장조사기관 IDC 가 예측한 AI 소프트웨어 시장의 연평균 성장률은 어떻게 되나요?"**

<br>

- **유사도 계산**
  - **1번: 80% -> 선택!**
  - 2번: 30%
  - 3번: 25%

<br>

<hr>

<br>

## `OpenAIEmbeddings`
- 문서 임베딩은 문서의 내용을 수치적인 벡터로 변환하는 과정. 이 과정을 통해 문서의 의미를 수치화하고, 다양한 자연어 처리 작업에 활용
- 대표적인 사전 학습된 언어 모델로는 BERT와 GPT가 있으며, 이러한 모델들은 문맥적 정보를 포착하여 문서의 의미를 인코딩
- 문서 임베딩은 토큰화된 문서를 모델에 입력하여 임베딩 벡터를 생성하고, 이를 평균하여 전체 문서의 벡터를 생성. 이 벡터는 문서 분류, 감성 분석, 문서 간 유사도 계산 등에 활용

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

- 지원되는 모델 목록
  - https://platform.openai.com/docs/guides/embeddings

<table><thead><tr><th>Model</th><th>~ Pages per dollar</th><th>Performance on <a target="_blank" rel="noopener noreferrer" href="https://github.com/embeddings-benchmark/mteb" class="kZ98Q" data-underline="">MTEB</a> eval</th><th>Max input</th></tr></thead><tbody><tr><td>text-embedding-3-small</td><td>62,500</td><td>62.3%</td><td>8192</td></tr><tr><td>text-embedding-3-large</td><td>9,615</td><td>64.6%</td><td>8192</td></tr><tr><td>text-embedding-ada-002</td><td>12,500</td><td>61.0%</td><td>8192</td></tr></tbody></table>

In [3]:
from langchain_openai import OpenAIEmbeddings

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

<br>

### 쿼리 임베딩
- 텍스트를 벡터 공간에 매핑하여 의미적으로 유사한 텍스트를 찾거나 텍스트 간의 유사도를 계산하는 데 사용

In [5]:
query_result = embeddings.embed_query(text)

query_result[:5]

[-0.007761634886264801,
 0.03675474599003792,
 0.019477618858218193,
 -0.019760848954319954,
 0.017146404832601547]

<br>

### Document 임베딩

In [6]:
doc_result = embeddings.embed_documents(
    [text]
)

In [11]:
doc_result[0][:5]

[-0.007778503466397524,
 0.03677903115749359,
 0.019533412531018257,
 -0.019729509949684143,
 0.017180250957608223]

<br>

### 차원 지정
- `text-embedding-3` 모델 클래스를 사용하면 반환되는 임베딩의 크기를 지정 가능
  - 기본적으로 `text-embedding-3-small`는 1536 차원의 임베딩을 반환

In [12]:
len(doc_result[0])

1536

- 임베딩의 크기를 1024로 축소
  

In [14]:
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

len(embeddings_1024.embed_documents([text])[0])

1024

<br>

### 유사도 계산

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [17]:
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [18]:
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [19]:
for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

[유사도 0.9644] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 반갑습니다!
[유사도 0.8376] 안녕하세요? 반갑습니다. 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.5042] 안녕하세요? 반갑습니다. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1362] 안녕하세요? 반갑습니다. 	 <=====> 	 I like to eat apples.
[유사도 0.8144] 안녕하세요? 반갑습니다! 	 <=====> 	 안녕하세요? 만나서 반가워요.
[유사도 0.4792] 안녕하세요? 반갑습니다! 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1318] 안녕하세요? 반갑습니다! 	 <=====> 	 I like to eat apples.
[유사도 0.5128] 안녕하세요? 만나서 반가워요. 	 <=====> 	 Hi, nice to meet you.
[유사도 0.1409] 안녕하세요? 만나서 반가워요. 	 <=====> 	 I like to eat apples.
[유사도 0.2249] Hi, nice to meet you. 	 <=====> 	 I like to eat apples.


<br>

<hr>

<br>

### 캐시 임베딩(`CacheBackedEmbeddings`)
- Embeddings는 재계산을 피하기 위해 저장되거나 일시적으로 캐시될 수 있음
- 캐시 지원 embedder는 embeddings를 키-값 저장소에 캐싱하는 embedder 주변에 래퍼. 텍스트는 해시되고 이 해시는 캐시에서 키로 사용

<br>

- `CacheBackedEmbeddings`를 초기화하는 주요 지원 방법은 `from_bytes_store`
  - `underlying_embeddings`: 임베딩을 위해 사용되는 embedder
  - `document_embedding_cache`: 문서 임베딩을 캐싱하기 위한 ByteStore 중 하나
  - `namespace`: (선택 사항, 기본값은 "") 문서 캐시를 위해 사용되는 네임스페이스. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용
    - 예) 사용된 임베딩 모델의 이름으로 설정
- **동일한 텍스트가 다른 임베딩 모델을 사용하여 임베딩될 때 충돌을 피하기 위해 namespace 매개변수를 설정하는 것이 중요**

<br>

### `LocalFileStore` 에서 임베딩 사용 (영구 보관)
- 로컬 파일 시스템을 사용하여 임베딩을 저장하고 FAISS 벡터 스토어를 사용하여 검색

In [30]:
import hashlib
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

In [47]:
def sha256_encoder(data) -> str:
    if isinstance(data, str):
        data = data.encode("utf-8")
    return hashlib.sha256(data).hexdigest()

In [48]:
embedding = OpenAIEmbeddings()
store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    # namespace=embedding.model,
    key_encoder=sha256_encoder,   # ★ SHA-256 적용
)

In [49]:
list(store.yield_keys())

['llm_cache.db', 'sqlite.db']

- 문서를 로드하고, 청크로 분할한 다음, 각 청크를 임베딩하고 벡터 저장소에 로드

In [50]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [51]:
raw_documents = TextLoader("./data/appendix-keywords.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

documents = text_splitter.split_documents(raw_documents)

- 문서로부터 FAISS 데이터베이스 생성

In [52]:
%time db = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 62.5 ms
Wall time: 1.79 s


- 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성 $\rightarrow$ **임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리**

In [54]:
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: total: 15.6 ms
Wall time: 12 ms


<br>

<hr>